## Economic Growth Model

The Solow Growth Model, developed by Robert Solow in the 1950s, is a cornerstone of theoretical economics, providing insights into the determinants of economic growth and the role of capital accumulation, labor force, and technological progress.

### The Basic Model

The Solow model posits that the output of an economy (Y) depends on the amounts of capital (K) and labor (L), along with a constant representing the level of technology (A). The model is often expressed with a production function that has constant returns to scale, typically a Cobb-Douglas production function:

$$ Y = A K^\alpha L^{1-\alpha} $$

where:
- \( Y \) is the total output (or real GDP) of the economy.
- \( A \) is a factor productivity term representing technology.
- \( K \) is the total amount of physical capital in the economy.
- \( L \) is the total labor force.
- \( \alpha \) (0 < α < 1) is the output elasticity of capital, indicating the percentage increase in output resulting from a percentage increase in capital, holding labor constant.

### Simplified Assumptions

To simplify the analysis, the Solow model often assumes that labor grows at a constant exogenous rate and that savings and investment are a fixed proportion of output. The model shows how these investments affect the capital stock over time, leading to economic growth through capital accumulation. The model also explores the concept of steady-state equilibrium, where the capital stock remains constant over time when depreciation and capital formation are balanced.

### Implications

One of the key insights from the Solow model is that higher savings rates can lead to a higher level of steady-state capital and output, but not to a higher growth rate in the long run. The long-term growth rate is determined by technological progress, according to the Solow model.

The model highlights the importance of technological advances and productivity improvements in sustaining long-term economic growth, beyond just the accumulation of capital and labor inputs.


## Hyman Minsky's Financial Instability Hypothesis and Minsky Moments

Hyman Minsky, an American economist, developed the Financial Instability Hypothesis (FIH), which emphasizes the dynamics of financial markets and their role in shaping economic cycles. His theory proposes that financial systems inherently tend toward periods of boom and bust due to the cyclical fluctuations in investor behavior and economic stability.

### Financial Instability Hypothesis

Minsky's model is built on the observation that during prosperous times, firms and consumers alike tend to increase their debt, becoming progressively riskier in their financial behaviors. This leads to what Minsky identified as three types of borrowers:
- **Hedge Borrowers:** Can pay back both the interest and principal from their cash flows.
- **Speculative Borrowers:** Can cover the interest but must continually roll over the principal.
- **Ponzi Borrowers:** Cannot cover the interest or principal from their operations and must borrow further or sell assets to meet their obligations.

As the economy grows, the proportion of speculative and Ponzi borrowers increases, elevating the financial risk within the system.

### Minsky Moments

A "Minsky Moment" is a sudden market collapse that follows a long period of bullish growth, typically caused by the unsustainable accumulation of debt by speculative and Ponzi borrowers. When these borrowers fail to meet their debt obligations due to a downturn or tightening credit conditions, it can lead to a rapid deflation of asset prices, increased volatility, and a retrenchment of economic activity.

These moments are characterized by:
- Rapid deleveraging
- Falling asset prices
- Abrupt slowdowns in economic activity

### Implications

The FIH and the concept of Minsky Moments have become particularly relevant in discussions of economic policy and regulation, especially following the global financial crisis of 2007-2008, which many analysts consider a quintessential Minsky Moment. Minsky's work suggests that to prevent such crises, regulatory measures should focus on curtailing excessive debt accumulation and managing the cyclical expansions of financial behavior.

Minsky's insights challenge the traditional views of financial markets always moving toward equilibrium, emphasizing instead the potential for sudden and severe economic instability due to internal market dynamics.


## Data Acquisition and Preprocessing Overview

This project involves analyzing economic and financial data to detect potential asset price bubbles. The data sources include GDP and job openings from governmental sources like the Federal Reserve Economic Data (FRED), the United States Census Bureau (USCB), and the Bureau of Labor Statistics (BLS), as well as stock market indices from various global markets.

### File Paths and Initial Data Loading

- **File Paths:** The project's data files are stored in a structured directory. The paths are set relative to a base path defined at the beginning of the script.
- **Loading Data:** The GDP and country keys data are loaded into pandas dataframes from CSV files. The GDP data includes multiple series, which are filtered based on a selected country.

### Specific Column Selection

- **Country and Series Identification:** A specific country, in this case, the United States, is selected, and corresponding series IDs for GDP and the stock index are extracted from the country keys data.
- **Data Filtering:** The main dataframe is filtered to include only the relevant GDP series and the 'USJO' column, which represents job openings data sourced from the BLS.

### Data Cleaning

- **Handling Missing Values:** The dataframe is cleaned by dropping any rows with missing values to ensure the integrity of the analyses.
- **Date Conversion:** The 'DATE' column is converted to datetime format, which facilitates time-series analysis and merging operations with other time-based data sources.

### Function for Data Selection

- **`get_observations_before_date_v3`:** This function extracts a specific number of observations from the dataframe before a given date. It ensures that the data is sorted and that only the relevant observations are returned, which is crucial for time-sensitive financial analyses.

### Stock Market Data Acquisition

- **Stock Index Data:** Stock data for the selected country’s primary index is downloaded using the `yfinance` library, which provides access to historical market data. The data is resampled to a quarterly frequency to match the GDP and job openings data.

### Data Merging

- **As-Of Merge:** The economic data (GDP and job openings) is merged with the stock market data using a backward as-of merge. This method aligns each stock market observation with the latest available economic data observation as of each date in the stock data.

### Data Alignment and Preparation for Analysis

- **Trimming and Alignment:** The length of the arrays containing calculated statistics (like GSADF statistics) is adjusted to match the length of the merged dataframe. This involves either trimming excess data or padding with NaNs to account for missing periods.
- **Final Merging:** The adjusted GSADF statistics are added to the merged dataframe, creating a comprehensive dataset ready for further analysis.


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from datetime import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from scipy.stats import norm
from arch import arch_model
from statsmodels.tsa.vector_ar.vecm import select_order
from statsmodels.tsa.vector_ar.vecm import select_coint_rank
from statsmodels.tsa.vector_ar.vecm import VECM, coint_johansen
from statsmodels.tsa.vector_ar.vecm import VECM
import os

import mplcyberpunk
plt.style.use("cyberpunk")
# python 3.9.12 for cyberpunk plot style 

In [ ]:
# Define file paths

series_list = '/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/Data/Series_List.csv'
pop_projections = '/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/Data/US_Census_Pop_Projections.csv'
df = pd.read_csv(series_list)
countries = pd.read_csv('/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/Data/Country Keys.csv')

# Correctly select specific columns
country = countries[['Country', 'GDP_SeriesID', 'Stock_index']]
# Example: Selecting the GDP_SeriesID for the USA
select = 'United States'

gdp_series_id = country.loc[country['Country'] == select, 'GDP_SeriesID'].values[0]
title = country.loc[country['Country'] == select, 'Country'].values[0]

# Assuming 'df' contains columns like 'DATE' and many GDP series
df = df[['DATE', gdp_series_id, 'USE' , 'USJO', 'USU', 'USP', 'Lockdowns']]#, 'USJO' ,'USU']]  # Selecting DATE and the specific GDP series column
df = df.rename(columns={gdp_series_id: 'GDP'})  # Rename the GDP series column

# Clean up data
df = df.dropna()

print(df)


In [ ]:
df1 = pd.read_csv(pop_projections)
df1

In [ ]:
def get_observations_before_date_v3(df, date_column, date_str, num_observations):
    """
    Extracts a specified number of observations from a DataFrame immediately before a given date using a date column.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        date_column (str): The name of the column that contains the date values.
        date_str (str): The target date in 'YYYY-MM-DD' format.
        num_observations (int): The number of observations to retrieve before the date.

    Returns:
        pd.DataFrame: A subset of the original DataFrame with the specified number of observations before the date.
    """
    # Convert the date column to datetime if not already
    df[date_column] = pd.to_datetime(df[date_column])

    # Sort the DataFrame by the date column to ensure chronological order
    df.sort_values(by=date_column, inplace=True)

    # Filter the DataFrame to only include rows before the target date
    before_date_df = df[df[date_column] < pd.to_datetime(date_str)]

    # Select the last 'num_observations' entries from this filtered DataFrame
    if len(before_date_df) > num_observations:
        df_subset = before_date_df.iloc[-num_observations:]
    else:
        df_subset = before_date_df  # In case there are fewer rows than 'num_observations'

    return df_subset
cut_date = '2016-10-31'
subset_df = get_observations_before_date_v3(df, 'DATE', cut_date, 295)

#df = subset_df

df 

First, pick a stock you would like to forcast from yahoo finance. For the purposes of demonstration we will use quarterly data of the DJIA 

In [ ]:
# Read stock data from yfinance 
# ^IXIC Nasdaq
# ^GSPC S&P
# ^DJA Dow
# Brazil Index ^BVSP
# India Index  ^BSESN
# Germany ^GDAXI
# Japan ^N225
# Mexico ^MXX
ticker = country.loc[country['Country'] == select, 'Stock_index'].values[0]
start = '1980-01-01'
end = df['DATE'].iloc[-1]
data = yf.download(ticker, start, end, interval='3mo')['Close']
T = len(data)
data_df = data.reset_index()
data_df.rename(columns={'Close': 'Price'}, inplace=True)

data_df

In [ ]:
merged_df = pd.merge_asof(df.sort_values('DATE'), 
                          data_df.sort_values('Date'), 
                          left_on='DATE', 
                          right_on='Date', 
                          direction='backward')
merged_df = merged_df.dropna()
merged_df

In [ ]:
price = data_df['Price']
price 
T = len(price)

## Overview of Explosive Unit Root Analysis Using SADF and GSADF Tests

The provided Python code implements the SADF and GSADF tests to detect explosive behavior (unit roots) in time series data. These tests are useful for identifying speculative bubbles or other forms of non-stationarity within economic data, which might indicate periods of excessive price increases followed by abrupt corrections.

### Initial Settings

- **`r0`**: The initial proportion of the sample used for the minimum window size for the rolling tests. It is set based on a formula that adjusts with the square root of the sample size `T`.
- **`swindow0`**: The actual size of the initial window converted from the proportion `r0`.
- **`dim`**: The number of test statistics that will be calculated, based on the total sample size `T` minus the size of the initial window `swindow0`.
- **`date`**: An array of dates that correspond to the length of the data, assuming quarterly frequency starting from the first index of the merged dataset.

### Functions

- **`simulate_critical_values(num_simulations, T, r0)`**: This function simulates critical values for the SADF/GSADF tests. It does so by repeatedly generating a unit root process and calculating the ADF statistic for each simulation. The critical values for the 90%, 95%, and 99% confidence levels are then derived from these simulated statistics.

### Calculation of SADF and GSADF Statistics

- **SADF Calculation**:
  - The SADF statistic is computed using a rolling window approach where the ADF test is applied to expanding windows of data starting from the initial window size to the end of the series.
  - The maximum ADF statistic from these windows is taken as the SADF statistic, indicating the strongest evidence of a unit root within any window.

- **GSADF Calculation**:
  - Similar to the SADF, but the GSADF allows for both starting and ending points of the window to vary.
  - This test computes the ADF statistic for every possible window combination within the dataset, starting from the initial window size.
  - The maximum of these statistics is the GSADF statistic, providing a more flexible test that can detect multiple periods of explosive behavior.

### Output

- The code outputs the calculated SADF and GSADF statistics and compares them against the critical values derived from the simulations. It also prints the length of the GSADF statistics array and the corresponding dates, which helps in identifying the specific time periods of potential explosive behavior.

### Usage

- The code is set up to analyze a series (likely stock prices or similar financial data) stored in the variable `price`.
- Users can adjust the number of simulations and the sample size `T` as needed based on their specific dataset and the desired accuracy of the test results.


In [ ]:
# Initial settings for the SADF test window
r0 = 0.01 + 1.8 / np.sqrt(T)
swindow0 = int(np.floor(r0 * T))
dim = T - swindow0 + 1
date = pd.date_range(start= merged_df.index[0], periods=T, freq='Q-DEC')

def simulate_critical_values(num_simulations, T, r0):
    critical_values = []
    for _ in range(num_simulations):
        # Generate a unit root process
        series = np.random.normal(size=T).cumsum()
        # Placeholder for SADF/GSADF calculation: this should ideally be replaced
        # with a function that calculates the SADF/GSADF statistic.
        stat = adfuller(series, maxlag=int(r0*T), regression='c', autolag=None)[0]
        critical_values.append(stat)
    
    # Determine the 90%, 95%, and 99% critical values
    critical_values = np.percentile(critical_values, [90, 95, 99])
    return critical_values

# Example usage
num_simulations = 2000

critical_values = simulate_critical_values(num_simulations, T, r0)
print("Critical values at 90%, 95%, 99%:", critical_values)


In [ ]:
# Calculate the SADF statistic
dateS = date[swindow0:]

badfs = np.zeros(dim)
for i in range(swindow0, T):
    result = adfuller(price[:i+1], maxlag=2, regression='c', autolag=None)
    badfs[i - swindow0] = result[0]
sadf = np.max(badfs)

print('The SADF statistic:', sadf)

In [ ]:
# Calculate the Generalized SADF statistic
bsadfs = np.zeros(dim)
for r2 in range(swindow0, T):
    dim0 = r2 - swindow0 + 1
    rwadft = np.zeros(dim0)
    for r1 in range(dim0):
        result = adfuller(price[r1:r2+1], maxlag=2, regression='c', autolag=None)
        rwadft[r1] = result[0]
    bsadfs[r2 - swindow0] = np.max(rwadft)

gsadf = np.max(bsadfs)

print('The GSADF statistic:', gsadf)
print('The critical values:', critical_values)
bsadfs = bsadfs[:-1]
len(bsadfs),len(dateS)


In [ ]:
# Assuming bsadfs, merged_df, and other necessary variables are already defined
# Print current lengths to understand the situation
print("Length of dateS:", len(dateS))
print("Length of bsadfs before trimming:", len(bsadfs))
print("Length of badfs:", len(badfs))

# Trim bsadfs if it's longer than merged_df
if len(bsadfs) > len(merged_df):
    bsadfs = bsadfs[-len(merged_df):]  # Trim from the beginning to match the end
elif len(bsadfs) < len(merged_df):
    # Calculate the number of missing periods
    missing_periods = len(merged_df) - len(bsadfs)
    # Create an array of NaNs for the missing periods
    initial_nans = np.full(missing_periods, np.nan)
    # Concatenate the NaNs with the bsadfs array to align with the end of merged_df
    bsadfs = np.concatenate((initial_nans, bsadfs))

# Add the bsadfs array to the DataFrame
merged_df['GSADF_Stat'] = bsadfs

# Optional: Drop rows with NaNs in the 'GSADF_Stat' if needed
merged_df = merged_df.dropna(subset=['GSADF_Stat'])
print(merged_df)


In [ ]:
merged_df = merged_df.drop(columns=['Date'])

# Display the DataFrame to verify the column has been removed
print(merged_df)

In [ ]:

def add_transformations(df, columns):
    """
    Add log, first difference, and log difference transformations to specified columns.

    Parameters:
    df (DataFrame): The DataFrame to modify.
    columns (list of str): List of column names to transform.

    Returns:
    DataFrame: The modified DataFrame with new columns.
    """
    for col in columns:
        # Calculate log of the column
        df[f'Log_{col}'] = np.log(df[col])

        # Calculate first difference of the column
        df[f'{col}_Diff'] = df[col].diff()

        # Calculate first difference of the log of the column
        df[f'Log_{col}_Diff'] = df[f'Log_{col}'].diff()

    # Drop the initial NaN values resulting from differencing
    df = df.dropna()

    return df

# Define the columns you want to transform
columns_to_transform = ['GDP', 'Price', 'USE', 'USJO', 'USU', 'USP']

# Apply the transformations
merged_df.set_index('DATE', inplace=True)
merged_df = add_transformations(merged_df, columns_to_transform)

# Display the resulting merged DataFrame
merged_df


In [ ]:
merged_df.to_csv('/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/Data/vecm_data.csv', index=True)  # creating csv of this data


In [ ]:
# Define color variables
color_job_openings = 'cyan'
color_bsadf_stat = 'red'
color_stock_index = 'limegreen'
color_gdp = 'yellow'
color_population = 'blue'
color_unemployment = 'magenta'
color_employment = 'orange'
axis_color = 'white' 

In [ ]:
merged_df['Log_USE']

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(merged_df.index, merged_df['Log_USE'], label='Log USE', marker='o', color=color_employment)
plt.plot(merged_df.index, merged_df['Log_USP'], label='Log USP', marker='o', color=color_population)

plt.title(f'{select} Job Openings and Unemployment', color=axis_color, fontsize=18, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Logarithmic Value')
plt.legend()
plt.grid(True)

plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}Pop_Growth & Employment.jpeg', format='jpeg')

plt.show()

## Importance of Checking Cointegration, Seasonality, and Stationarity in VECM Specification

When specifying a Vector Error Correction Model (VECM), it is crucial to assess cointegration, seasonality, and stationarity of the involved time series. These checks are fundamental for ensuring the validity of the model, the accuracy of the inferences drawn from it, and the effectiveness of the model in capturing the underlying economic dynamics.

### 1. Stationarity

- **Definition:** A stationary time series has a constant mean, variance, and autocorrelation structure over time, meaning its statistical properties do not change over time.
- **Importance in VECM:**
  - **Model Stability and Reliability:** Non-stationary data can lead to spurious results in regression-type models, which assume stationary inputs. VECM requires that all series included in the model are integrated of the same order, typically I(1), which means they become stationary after first differencing.
  - **Pre-condition for Cointegration:** Only non-stationary series that are integrated of the same order can be tested for cointegration, a key component in the VECM framework.

### 2. Cointegration

- **Definition:** Cointegration occurs when a linear combination of two or more non-stationary series is itself stationary. This implies a long-term equilibrium relationship among the series despite short-term deviations.
- **Importance in VECM:**
  - **Capturing Long-Term Relationships:** Cointegration indicates that the variables share a common stochastic trend and are bound by an equilibrium relationship, even though they may individually wander away from equilibrium.
  - **Error Correction Term (ECT):** VECM includes an error correction term derived from the cointegration equation, which adjusts the short-term dynamics of the dependent variables to correct for deviations from this long-term equilibrium.

### 3. Seasonality

- **Definition:** Seasonality refers to periodic fluctuations in time series data that occur at regular intervals, such as quarterly, monthly, or yearly.
- **Importance in VECM:**
  - **Model Accuracy:** Ignoring seasonal variations can lead to misinterpretations of trends and cycles in the data. Seasonal adjustments ensure that the model captures true underlying trends rather than seasonal effects.
  - **Improved Forecasting:** Correctly modeling seasonality improves the model's predictive performance, particularly for economic and financial data, which often exhibit strong seasonal patterns.


In [ ]:
def perform_adf_test(series, regression_type='ct'):
    """
    Perform an Augmented Dickey-Fuller test on a given time series.

    Parameters:
        series (pd.Series): The time series on which to perform the ADF test.
        regression_type (str): The type of regression ('c' for constant, 'ct' for constant and trend, 'ctt' for constant, and linear and quadratic trend, 'nc' for no constant, no trend).

    Returns:
        None
    """
    result = adfuller(series, regression=regression_type)
    print(f"ADF Statistic for {series.name}: {result[0]}")
    print(f"p-value for {series.name}: {result[1]}")
    print("Critical Values:")
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')

    if result[0] < result[4]['10%']:
        print(f"Reject the null hypothesis - the {series.name} series is stationary at the 90% confidence level.")
    else:
        print(f"Fail to reject the null hypothesis - the {series.name} series is not stationary at the 90% confidence level.")


In [ ]:
for column in merged_df.columns:
    result = perform_adf_test(merged_df[column], regression_type='ct')
    print(f'ADF Test for {column}:\n{result}\n')

In [ ]:
# Plot Correlogram for Log Stock Index Price
plt.figure(figsize=(10, 5))
plot_acf(merged_df['Log_Price'], lags=40, zero=False, title='Autocorrelation of Log Stock Index Price', color=color_stock_index)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot Correlogram for Log GDP
plt.figure(figsize=(10, 5))
plot_acf(merged_df['Log_GDP'], lags=40, zero=False, title='Autocorrelation of Log National GDP', color=color_gdp)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot Correlogram for Log US Job Openings
plt.figure(figsize=(10, 5))
plot_acf(merged_df['Log_USJO'], lags=40, zero=False, title='Autocorrelation of Log National Job Openings', color=color_job_openings)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot Correlogram for Log Unemployment
plt.figure(figsize=(10, 5))
plot_acf(merged_df['Log_USU'], lags=40, zero=False, title='Autocorrelation of Stock Index Price', color=color_unemployment)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot Correlogram for Log Unemployment
plt.figure(figsize=(10, 5))
plot_acf(merged_df['Log_USP'], lags=40, zero=False, title='Autocorrelation of Population', color=color_population)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Plot Correlogram for GSADF Statistic
plt.figure(figsize=(10, 5))
plot_acf(merged_df['GSADF_Stat'], lags=40, zero=False, title='Autocorrelation of GSADF Statistic', color=color_bsadf_stat)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

In [ ]:

# Create the plot with specified figure size
plt.figure(figsize=(14, 7))
ax1 = plt.gca()  # Get the current axes instance

# Plotting Job Openings on the left y-axis
ax1.plot(merged_df.index, merged_df['Log_USJO'], label='Log of US Job Openings', color=color_job_openings)
ax1.plot(merged_df.index, merged_df['Log_USU'], label='Log of US Unemployment', color=color_unemployment)

ax1.set_xlabel('Date')
ax1.set_ylabel('Log of US Job Openings', color=color_job_openings)
ax1.tick_params(axis='y', colors=axis_color)
ax1.grid(False)  # Disable gridlines for the primary axis

# Create a second y-axis for the BSADF Statistic data
ax2 = ax1.twinx()
ax2.plot(merged_df.index, merged_df['GSADF_Stat'], label='Test Statistic', color=color_bsadf_stat)
ax2.set_ylabel('BSADF Statistic', color=color_bsadf_stat)
ax2.tick_params(axis='y', colors=axis_color)
ax2.grid(False)  # Disable gridlines for the secondary axis

# Title and legend handling
plt.title(f'{select} Employment Statistics and Stock Market Bubbles over Time', color=axis_color, fontsize=18, fontweight='bold')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')  # Adjust location as needed

# Save the plot to the specified directory and file
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}_Unemployment_Test_Stat.jpeg', format='jpeg')

# Show the plot
plt.show()


In [ ]:
# Create a figure and a set of subplots with the desired size
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot the Price data on the primary y-axis
ax1.set_xlabel('Date')
ax1.set_ylabel('Stock Index', color=color_stock_index)
ax1.plot(merged_df.index, merged_df['Log_Price'], color=color_stock_index, label='Log Stock Index')
ax1.tick_params(axis='y', labelcolor=axis_color)
ax1.grid(False)  # Disable gridlines for the primary axis

# Create a second y-axis for the GDP and US Prices data
ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
ax2.set_ylabel('GDP and US Prices', color=color_gdp)  # We already handled the x-label with ax1
ax2.plot(merged_df.index, merged_df['Log_GDP'], color=color_gdp, label='Log GDP')
ax2.plot(merged_df.index, merged_df['Log_USP'], color=color_population, linestyle='--', label='Log US Population')  # Corrected label
ax2.tick_params(axis='y', labelcolor=axis_color)
ax2.grid(False)  # Disable gridlines for the secondary axis

# Title and layout
plt.title('Stock Index, National GDP, and US Prices Over Time')
fig.tight_layout()  # Otherwise the right y-label is slightly clipped

# Add legend to differentiate the data series
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')  # Adjust location as needed
plt.title(f'{select} Capital Stock, Output and Population Growth over Time', color=axis_color, fontsize=18, fontweight='bold')

# Save the plot to the specified directory and file
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}_Index_Price_GDP_USP.jpeg', format='jpeg')

# Show the plot
plt.show()


In [ ]:
# Customize plotting function
def customize_plot(result, title, color):
    fig, axes = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(12, 8))
    axes[0].plot(result.observed, color=color)
    axes[0].set_title('Observed')
    
    axes[1].plot(result.trend, color=color)
    axes[1].set_title('Trend')
    
    axes[2].plot(result.seasonal, color=color)
    axes[2].set_title('Seasonal')
    
    axes[3].plot(result.resid, color=color)
    axes[3].set_title('Residual')
    
    for ax in axes:
        ax.tick_params(axis='x', colors=axis_color)
        ax.tick_params(axis='y', colors=axis_color)

    plt.suptitle(title, color=axis_color)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    

# Applying the function to each series
# Decompose and plot Stock Index data
result_price = seasonal_decompose(merged_df['Price'], model='additive', period=4)  
customize_plot(result_price, 'Stock Index Decomposition', color_stock_index)

# Decompose and plot GDP data
result_gdp = seasonal_decompose(merged_df['GDP'], model='additive', period=4)  
customize_plot(result_gdp, 'GDP Decomposition', color_gdp)

# Decompose and plot Unemployment data
result_job_openings = seasonal_decompose(merged_df['USU'], model='additive', period=4)  
customize_plot(result_job_openings, 'Unemployment Decomposition', color_unemployment)
# Decompose and plot Job Openings data
result_job_openings = seasonal_decompose(merged_df['USJO'], model='additive', period=4)  
customize_plot(result_job_openings, 'Job Openings Decomposition', color_job_openings)

# Decompose and plot Population data
result_population = seasonal_decompose(merged_df['USP'], model='additive', period=4)  
customize_plot(result_population, 'Population Decomposition', color_population)
# Decompose and plot GSADF data
result_gsadf = seasonal_decompose(merged_df['GSADF_Stat'], model='additive', period=4)  
customize_plot(result_gsadf, 'GSADF Stat Decomposition', color_bsadf_stat)

merged_df.columns


## Vector Error Correction Model (VECM) with Explosive Unit Root, Job Openings, Stock Market Growth, and GDP

The Vector Error Correction Model (VECM) is a multivariate statistical model used to analyze the long-term and short-term dynamics among several non-stationary time series variables that are cointegrated. In this model, we include variables such as an explosive unit root, job openings, stock market growth, and GDP to understand their interdependencies and the adjustments toward equilibrium over time.

### Overview of the VECM

A VECM is particularly useful when dealing with non-stationary data series that exhibit a long-run equilibrium relationship, as suggested by cointegration tests. The inclusion of the error correction term allows the model to specify both the short-term dynamics and adjustments needed to return to equilibrium after shocks.

### Components of the Model

- **Explosive Unit Root:** This component typically represents a variable that shows persistent exponential growth or decline, diverging from a stable path. In financial data, this might be related to speculative bubbles or severe recessive movements.
- **Job Openings (USJO):** This measures the labor demand and is an indicator of economic health. Fluctuations in job openings can be both a cause and effect of economic cycles.
- **Stock Market Growth:** Often a leading indicator of economic health, as it reflects investor expectations about future corporate earnings and macroeconomic trends.
- **GDP (Gross Domestic Product):** Represents the total economic output and is a crucial measure of overall economic activity and health.

### Functioning of the VECM

In the VECM:
- The **long-term relationship** is captured by the cointegration among the variables, indicating that despite short-term fluctuations, the variables move together over time towards an equilibrium.
- The **short-term dynamics** are detailed by the coefficients of the lagged differences of the variables, indicating how each variable adjusts to changes in the others in the short term.
- The **error correction term** (ECT), derived from the deviation from the long-run equilibrium in the previous period, measures the speed and direction of adjustment back towards the equilibrium.

### Applications and Implications

This model can be particularly insightful for policymakers and investors as it provides:
- Insights into how quickly economic variables return to stability after a shock, which is crucial for monetary and fiscal policy decisions.
- Understanding the impact of stock market fluctuations and job market dynamics on overall economic growth.
- Early warnings of economic overheating or underperformance through the explosive unit root component, guiding preemptive actions.

By analyzing the interactions and adjustments between these critical economic indicators, a VECM helps in predicting future economic conditions and designing informed economic policies.


In [ ]:
# Fit the model
# Determine the optimal lag order
data = merged_df[['Log_Price', 'Log_GDP', 'Log_USJO','Log_USU','GSADF_Stat','Log_USP']]

lag_order = select_order(data=data, maxlags=20, deterministic="ci")
print(lag_order.summary())
optimal_lags = lag_order.fpe
# Override if still overfit.  Lag length is in large part a judgement call
optimal_lags =  7
optimal_lags

In [ ]:
# Perform the Johansen cointegration test
data = merged_df[['Log_Price', 'Log_GDP', 'Log_USJO','Log_USU','GSADF_Stat','Log_USP', 'Lockdowns']]

result = coint_johansen(data[['Log_Price', 'Log_GDP', 'Log_USJO','Log_USU','GSADF_Stat','Log_USP']], det_order=0, k_ar_diff= optimal_lags)

# Print the Trace Statistics and Critical Values
print('Trace Statistics:', result.lr1)
print('Critical Values (90%, 95%, 99%):', result.cvt)

# Interpret and print whether cointegration is found
cointegration_found = False  # A flag to keep track of cointegration status
for i, trace_stat in enumerate(result.lr1):
    if trace_stat > result.cvt[i][0]:  # Index 1 is for the 95% confidence level
        cointegration_found = True
        print(f"At the 90% confidence level, we reject the null hypothesis of at most {i} cointegrating relations; indicating cointegration.")
    else:
        print(f"At the 90% confidence level, we fail to reject the null hypothesis of at most {i} cointegrating relations; no cointegration indicated.")

# Print overall result based on tests
if cointegration_found:
    print("The series exhibit a long-run cointegrating relationship.")
else:
    print("No long-run cointegrating relationship is evident among the series.")


In [ ]:
rank_test = select_coint_rank(data[['Log_Price', 'Log_GDP', 'Log_USJO','Log_USU','GSADF_Stat','Log_USP']], 0, optimal_lags, method="trace",
                              signif=0.10)
rank_test.summary()


## Understanding the Results of the `select_rank` Function

The `select_rank` function is an essential tool in time series analysis, especially when setting up a Vector Error Correction Model (VECM). This function helps in determining the number of cointegrating relationships among a set of non-stationary variables. Here's a breakdown of what the results mean and how they are interpreted:

### Purpose of `select_rank`

- **Identify Cointegration Rank:** The main purpose of the `select_rank` function is to identify the rank of cointegration among the variables in a given dataset. The cointegration rank specifies the number of linearly independent vectors that combine the integrated variables into stationary processes.

### How It Works

- **Test Setup:** The function typically utilizes tests like the Johansen cointegration test, which examines the null hypothesis of `r` cointegrating relationships against the alternative of more cointegrating vectors up to the number of included variables.
- **Statistical Methods:** It involves methods such as trace statistics and maximum eigenvalue statistics, which help in determining the presence and number of cointegrating relationships.

### Results Interpretation

1. **Number of Cointegrating Vectors (Rank)**
   - The function outputs the estimated number of cointegrating relationships. This number tells us how many independent long-term relationships exist among the variables.
   - **Example:** If `select_rank` returns a cointegration rank of 2 for a system of 5 variables, it implies there are two unique long-term equilibrium relationships binding these variables together.

2. **Statistical Values**
   - **Trace Statistic and Maximum Eigenvalue:** These are the key outputs used to determine the cointegration rank. Each statistic tests for additional cointegrating vectors.
   - The results typically include critical values or p-values, which help in deciding whether to reject the null hypothesis of `r` cointegrating vectors.

3. **Model Specification**
   - Based on the rank, the error correction terms in the VECM are specified. Each cointegrating relationship will contribute one error correction term to the model, guiding how quickly variables adjust to restore equilibrium after a shock.



In [ ]:
df1.set_index('Date', inplace=True)


In [ ]:
# Function to round up the last three digits after dividing by 1000
def round_up_last_three(x):
    return np.ceil(x / 1000)

# Apply the function to each of the projection columns correctly
df1['US_Proj-lo'] = round_up_last_three(df1['US_Proj-lo']).astype(int)
df1['US_Proj-0'] = round_up_last_three(df1['US_Proj-0']).astype(int)
df1['US_Proj-hi'] = round_up_last_three(df1['US_Proj-hi']).astype(int)

# Show the updated DataFrame
print(df1)


In [ ]:
# Set 'Date' as the index if it's not already
df1.index = pd.to_datetime(df1.index, format='%Y')
df1 = df1[2:]

# Create a plot
plt.figure(figsize=(10, 5))  # Set the figure size for better visibility

# First Plot: Low Migration Scenario
plt.subplot(3, 1, 1)  # This means 3 rows, 1 column, first plot
plt.plot(df1['US_Proj-lo'], label='Low Migration Scenario', color='blue')
plt.title('Low Migration Scenario')
plt.ylabel('Population')
plt.legend()
plt.grid(True)
plt.ylim(200000, 450000)  # Set uniform y-axis limits

# Second Plot: High Migration Scenario
plt.subplot(3, 1, 2)  # This means 3 rows, 1 column, second plot
plt.plot(df1['US_Proj-hi'], label='High Migration Scenario', color='green')
plt.title('High Migration Scenario')
plt.ylabel('Population')
plt.legend()
plt.grid(True)
plt.ylim(200000, 450000)  # Set uniform y-axis limits

# Third Plot: Zero Migration Scenario
plt.subplot(3, 1, 3)  # This means 3 rows, 1 column, third plot
plt.plot(df1['US_Proj-0'], label='Zero Migration Scenario', color='red')
plt.title('Zero Migration Scenario')
plt.xlabel('Date')  # Only adding x-label to the bottom plot for clarity
plt.ylabel('Population')
plt.legend()
plt.grid(True)
plt.ylim(200000, 450000)  # Set uniform y-axis limits

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figure
plt.savefig('/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/Pop_Growth_Estimates.jpeg', format='jpeg')

# Show the plot
plt.show()

In [ ]:
# Assuming df1 has the 'Date' index as datetime at the start of each year
df1.index = pd.to_datetime(df1.index, format='%Y')

# Resample the DataFrame to quarterly, starting from the beginning of the year
pop_estimate_df = df1.resample('QS').asfreq()  # 'QS' stands for Quarter Start

# Apply interpolation
pop_estimate_df.interpolate(method='polynomial', order=3, inplace=True)

pop_estimate_df = np.log(pop_estimate_df)

# Check the resulting DataFrame
print(pop_estimate_df) # Show the last few rows to verify


In [ ]:
# Configure and fit the VECM model
vecm_model = VECM(endog = data[['Log_Price','Log_GDP','Log_USJO','Log_USU','GSADF_Stat']], 
                  exog_coint = data[['Log_USP']], 
                  k_ar_diff= optimal_lags, 
                  coint_rank=5, 
                  deterministic= 'ci')
vecm_result = vecm_model.fit()
vecm_result.summary()


Plotting GDP and Stock Market Predictions

In [ ]:
num_periods = 200
pop_estimate_df = pop_estimate_df.iloc[:num_periods]
pop_estimate_df


In [ ]:
num_periods = 200
pop_estimate_df = pop_estimate_df.iloc[:num_periods]

# Assuming `exog_forecasts_df` contains the exogenous forecasts with columns
# 'Log_USP_Forecast1', 'Log_USP_Forecast2', 'Log_USP_Forecast3'
pop_estimate_df['Log_USP'] = pop_estimate_df[['US_Proj-hi']]
pop_estimate = ['High Migration'] # adjust name of graphs
# Here we will just use the first column as an example
exog_coint_fc = pop_estimate_df[['Log_USP']].values

pop_estimate_df['Lockdowns'] = 0

exog_fc = pop_estimate_df['Lockdowns']

# Generate the forecasted data using the predict function
forecast_df = vecm_result.predict(steps=num_periods, exog_coint_fc=exog_coint_fc)

# Plotting the forecast
forecast_df = pd.DataFrame(forecast_df, columns=data[['Log_Price','Log_GDP','Log_USJO','Log_USU','GSADF_Stat']].columns, index=pop_estimate_df.index)

forecast_df

## Understanding Impulse Response Function (IRF) Plots from VECM

Impulse Response Function (IRF) plots are a crucial tool in time series analysis, particularly when using Vector Error Correction Models (VECM) to understand the dynamic interactions between variables. These plots illustrate how a shock to one variable affects other variables in the system over time. Here’s an explanation of how to interpret these plots and understand their implications.

### What are IRF Plots?

- **Definition:** IRF plots show the reaction of dependent variables in a VECM to one-time shocks in each of the other variables. They help in visualizing the time path of these effects and the gradual adjustment of the variables towards equilibrium.
- **Purpose:** The primary purpose of generating IRF plots in a VECM framework is to analyze the effect of changes (or shocks) in one variable on the entire system over a specified number of periods.

### Generating IRF Plots

- **Procedure:** In the provided code snippet, the impulse response function is computed for a given number of periods (`num_periods = 200`). This function simulates the response of all endogenous variables in the model to a shock in each variable, one at a time.
- **Visualization:** The method `ir.plot(plot_stderr=False)` generates a series of line plots. Each plot represents the response of one variable to a unit shock in each of the other variables, including itself, over the 200 periods.
- **File Saving:** The plots are then saved as JPEG images in a specified directory, allowing for further analysis or presentation.

### Interpretation of IRF Plots

1. **Response Over Time:**
   - Each line on the plot shows how a particular variable reacts over time after the shock. The x-axis represents time in periods after the shock, and the y-axis shows the magnitude of the variable's response.
   - Positive values indicate an increase in the variable relative to its baseline value due to the shock, while negative values indicate a decrease.

2. **Speed and Magnitude of Adjustment:**
   - The plots provide insights into how quickly and significantly variables adjust in response to shocks. Rapid declines or increases followed by stabilization suggest quick adjustments to shocks.
   - Large swings or sustained deviations from zero can indicate high sensitivity to shocks and potentially longer-term impacts on the variable.

3. **Economic Implications:**
   - By observing the IRF plots, analysts can infer the stability and resilience of the economic system or market under study. For example, a prolonged or significant response in GDP or stock prices to a policy change or economic event might signal underlying vulnerabilities or strong dependencies.


In [ ]:
ir = vecm_result.irf(periods=num_periods)
fig = ir.plot(plot_stderr=False)

# Adjust the font size of the labels
for ax in fig.axes:
    ax.title.set_fontsize(10)  # Adjust title font size
    ax.xaxis.label.set_fontsize(8)  # Adjust x-axis label font size
    ax.yaxis.label.set_fontsize(8)  # Adjust y-axis label font size
    ax.tick_params(axis='both', which='major', labelsize=8)  # Adjust tick labels font size

# Show the plot
plt.show()

filename = f'{ticker}_irf.jpg'
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select} {pop_estimate} IRF.jpeg', format='jpeg')



Forecast Level Calculation:

add the forecasted differences back to the last observed level, making the output interpretable in the original data's context.

In [ ]:
# Merge the forecasted data with the observed data
merged_df = pd.concat([data, forecast_df])


In [ ]:
# Create the main figure and axis
fig, ax1 = plt.subplots(figsize=(12, 8))


# Plotting all logarithmic data on the primary left y-axis
ax1.set_xlabel('Date', style='italic', color=axis_color, fontsize=12, fontweight='bold')
ax1.set_ylabel('Logarithmic Values', fontsize=12, fontweight='bold')
ax1.plot(merged_df.index, merged_df['Log_GDP'], color=color_gdp, label='GDP (Log)')
ax1.plot(merged_df.index, merged_df['Log_Price'], color=color_stock_index, label='DJIA Price (Log)')
ax1.plot(merged_df.index, merged_df['Log_USJO'], color=color_job_openings, label='Job Openings (Log)')
ax1.plot(merged_df.index, merged_df['Log_USU'], color=color_unemployment, label='Unemployment (Log)')
ax1.plot(merged_df.index, merged_df['Log_USP'], color=color_population, label='Population (Log)')  # New population line

# Optional: Forecast data plotting
ax1.plot(forecast_df.index, forecast_df['Log_GDP'], linestyle='-', color=color_gdp, label='GDP (Log)')
ax1.plot(forecast_df.index, forecast_df['Log_Price'], linestyle='-', color=color_stock_index, label='DJIA Price (Log)')
ax1.plot(forecast_df.index, forecast_df['Log_USJO'], linestyle='-', color=color_job_openings, label='Job Openings (Log)')
ax1.plot(forecast_df.index, forecast_df['Log_USU'], linestyle='-', color=color_unemployment, label='Unemployment (Log)')
ax1.plot(forecast_df.index, pop_estimate_df['Log_USP'], linestyle='-', color=color_population, label='Population (Log)') 

ax1.tick_params(axis='y', labelcolor=axis_color, labelsize=10)
ax1.grid(False)  # Disable gridlines for this axis

# Create a second y-axis for the GSADF Statistic data
ax2 = ax1.twinx()
ax2.set_ylabel('GSADF Statistic', fontsize=12, fontweight='bold')
ax2.plot(merged_df.index, merged_df['GSADF_Stat'], color=color_bsadf_stat, label='Explosivity Test Statistic')
ax2.plot(forecast_df.index, forecast_df['GSADF_Stat'], linestyle='-', color=color_bsadf_stat, label='Explosivity Test Statistic')
ax2.set_ylabel('Explosivity Test Statistic', fontsize=12, fontweight='bold')
ax2.grid(False)  # Disable gridlines for this axis

# Adding a vertical line for the last observed date
last_observed_date = forecast_df.index[0]  # Assuming forecast_df.index contains the dates
plt.axvline(x=last_observed_date, color='black', linestyle='--', linewidth=2, label='Last Observed Date')

ax2.axhline(y=critical_values[1], color='black', linestyle='--', linewidth=2, label='Explosive Unit Root Critical Value')

# Title and legend handling
plt.title(f'{select} Economic Indicators and Stock Market Bubbles over Time', color=axis_color, fontsize=18, fontweight='bold')
fig.tight_layout()

# Collecting handles and labels from both axes for a unified legend, avoiding duplicates
handles, labels = [], []
for ax in [ax1, ax2]:
    for handle, label in zip(*ax.get_legend_handles_labels()):
        if label not in labels:
            handles.append(handle)
            labels.append(label)
ax1.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.25,1.01), frameon = True)

# Save and show the plot
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}_{pop_estimate}_Model.jpeg', format='jpeg')
plt.show()


In [ ]:
T = len(merged_df)



In [ ]:
# Create the plot with specified figure size
plt.figure(figsize=(14, 7))
ax1 = plt.gca()  # Get the current axes instance

# Plotting Job Openings on the left y-axis
ax1.plot(merged_df.index, merged_df['Log_USJO'], label='Log of US Job Openings', color=color_job_openings)
ax1.plot(merged_df.index, merged_df['Log_USU'], label='Log of US Unemployment', color=color_unemployment)

ax1.set_xlabel('Date')
ax1.set_ylabel('Logarithmic Values', fontsize=12, fontweight='bold')
ax1.tick_params(axis='y', colors=axis_color)
ax1.grid(False)  # Disable gridlines for the primary axis

# Create a second y-axis for the BSADF Statistic data
ax2 = ax1.twinx()
ax2.plot(merged_df.index, merged_df['GSADF_Stat'], label='Explosivity Test Statistic', color=color_bsadf_stat)
ax2.set_ylabel('Explosivity Test Statistic', fontsize=12, fontweight='bold')
ax2.tick_params(axis='y', colors=axis_color)
ax2.grid(False)  # Disable gridlines for the secondary axis


# Title and legend handling
plt.title('Employment Statistics and Stock Market Bubbles over Time', color=axis_color, fontsize=18, fontweight='bold')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')  # Adjust location as needed

# Save the plot to the specified directory and file
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}_{pop_estimate} Unemployment_Test_Stat_F.jpeg', format='jpeg')

# Show the plot
plt.show()


In [ ]:
# Create a figure and a set of subplots with the desired size

# Create a figure and a set of subplots with the desired size
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot the Price data on the primary y-axis
ax1.set_xlabel('Date')
ax1.set_ylabel('Capital Stock', fontsize=12, fontweight='bold', color=color_stock_index)
line1, = ax1.plot(merged_df.index, merged_df['Log_Price'], color=color_stock_index, label='Dow Jones Industrial Average Prive')
ax1.tick_params(axis='y', labelcolor=axis_color)
ax1.grid(False)  # Disable gridlines for the primary axis

# Create a second y-axis for the GDP data
ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Output', fontsize=12, fontweight='bold', color=color_gdp)
line2, = ax2.plot(merged_df.index, merged_df['Log_GDP'], color=color_gdp, label='Gross Domestic Product')
ax2.tick_params(axis='y', labelcolor=axis_color)
ax2.grid(False)  # Disable gridlines for the secondary axis

ax3 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
ax3.spines['right'].set_position(('outward', 60))  # Offset the third axis
ax3.set_ylabel('Explosivity Test Statistic', fontsize=12, fontweight='bold', color=color_bsadf_stat)
line3, = ax3.plot(merged_df.index, merged_df['GSADF_Stat'], color=color_bsadf_stat, label='Explosivity Test Statistic')
ax3.tick_params(axis='y', labelcolor=axis_color)
ax3.grid(False)  # Disable gridlines for the tertiary axis

# Title and layout
plt.title('Capital Stock, Output and Stock Market Bubbles over Time', color=axis_color, fontsize=18, fontweight='bold')
fig.tight_layout()  # Otherwise the right y-label is slightly clipped

# Add a legend to the plot
lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='best')


# Save the plot to the specified directory and file
plt.savefig(f'/Users/philiplacava/Desktop/Asset-Price-Bubble-Detection/results/{select}_Index_Price_GDP_F.jpeg', format='jpeg')

# Show the plot
plt.show()
